<img align="right" width="125" src="https://www.ou.nl/documents/40554/3255217/Logo_OU.jpg"/>

<br>
<center> <font size ="6" color='red'> Production Planning Dashboard </font></center>
<center> <font size ="6" color='red'> Planning and Scheduling under uncertainty </font></center> <br>

<font size ="4" color='red'>*Introduction*</font> <br>
In this production planning dashboard, we consider three basic sets of objects; product, resources, and operations. A product is an output of an activity, so-called operation, that consumes certain capacity of the specified resource. A product has a set of predecessors that should be ready to start the operation of it. Every product and its predecessor relation is specified with a multiplier such that so many predecessors are needed to produce one of the product. For example, one simple electronic card may need four screws and one microcontroller to be assembled. Here, this electronic card has two predcessors; screw and microcontroller with multipliers four and one respectively. Note that if a product has no predecessor, then it is a raw material. Every raw material in our problem has an initial stock level and they are consumed in required amounts when the successor products start being produced. <br> 

Resources are of several types, e.g. machines and operators. An operation takes certain time to execute by a resource and it is in unit either machine-hour or man-hour. A customer order speficies the product in demand, the required quantity, and the desired date for the delivery of the products. <br> 

The goal of the production planning is to determine how much (time-dependent) capacity of every resource is consumed to complete the customer orders on time and how many raw materials are necessary for the operations.



<font size ="4" color='red'> *Baseline Planning Procedure:* </font><br>

In this assignment you will implement s baseline planning procedure. Orders have priorities due to their deadlines; i.e. the customer order with soonest deadline must be planned first. The primary goal is to plan customer orders without minumum delay. You will determine the delivery dates of customer orders starting from the one with earliest deadline and continue with the other with next-earliest deadline, and so on. For each customer order, you will determine the required (cumulative) capacity use by following the BOMs of the final product backwards as we have shown in the running example of simple flow shop. When an infeasibility in capacity level of any resource is encountered, then you will try to recover to the feasible case by delaying the order delivery date, increnetally in one day steps. Once a delay of certain days brings a feasible required capacity use, then you will continue planning in the BOM till all raw materials are reached. Then the requires stock levels of raw materials are determined, as we have shown in the running example. <br>   


The planning of customer orders will also require stock levels of raw materials. 
Every raw material can be purchased at most certain amount in every week. The required raw material levels of planned customer orders should be feasible by possibly purchasing additional amount of raw materials for every week. 


In [ ]:
import os
online_version = False;editmode = True
if online_version:  
    user = "muratfirat78";repo = "ProductionPlanning"
    if os.path.isdir(repo):
        !rm -rf {repo}
        
    !git clone https://github.com/{user}/{repo}.git
    %cd /content/{repo}
from DBMain import*
DataMgr.setOnlineVersion(online_version);VisMgr.setEditMode(editmode)
#############################################################################################################
tab_1 = VisMgr.get_case_selection_tab(); tab_4 = VisMgr.generatePSTAB(); tab_2 = VisMgr.generateCOTAB(); tab_3 = VisMgr.generatePLTAB()
tab_5 = VisMgr.generatePSschTAB()
####################################################################################################################
tab_set = widgets.Tab([tab_1,tab_4,tab_2,tab_3,tab_5])
tab_set.set_title(0, 'UseCase Selection'); tab_set.set_title(1, 'Production System'); tab_set.set_title(2, 'Customer Orders');
tab_set.set_title(3, 'Planning'); tab_set.set_title(4, 'Scheduling');tab_set

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import os

bom_df = pd.read_csv("Bill of Material (mrp.bom).csv")

prodname = "[00236-05-103-C] geleideblok spanrol" 
print(prodname in bom_df['Reference'].unique())

DataMgr.getResources().clear()
DataMgr.getProducts().clear()

def MakeProduct(dbmgr,prodinfo):

    prodname = prodinfo
    prodno = "XXXXXX"
    if (prodinfo.find("[")> -1 ) and (prodinfo.find("]")> -1 ):
        prodno = prodinfo[prodinfo.find("[")+1:prodinfo.find("]")]
        prodname = prodinfo[prodinfo.find("]")+1:]

    if not prodname+"_["+prodno+"]" in dbmgr.getProducts():
        newprod = Product(len(dbmgr.getProducts()),prodname+"_["+prodno+"]",prodno,0)
        return newprod

    else:
        return dbmgr.getProducts()[prodname+"_["+prodno+"]"]

def MakeOperation(dbmgr,prod,oprinfo):
    
    opname = oprinfo[:oprinfo.find("(")]
    opmach = oprinfo[oprinfo.find("(")+1:oprinfo.find(")")]

    

    myres = None
    if (oprinfo.find("(")> -1 ) and (oprinfo.find(")")> -1 ):
        
        if not opmach in dbmgr.getResources():
            myres = Resource(len(dbmgr.getResources()),"Machine",opmach,16)
            dbmgr.getResources()[opmach]=myres
        else: 
            myres = dbmgr.getResources()[opmach]
    else:
        if oprinfo.find(" - ") > -1:
            opname = oprinfo[:oprinfo.find(" - ")]
            opmach = oprinfo[oprinfo.find(" - ")+3:]
            if not opmach in dbmgr.getResources():
                myres = Resource(len(dbmgr.getResources()),"Machine",opmach,16)
                dbmgr.getResources()[opmach]=myres
            else: 
                myres = dbmgr.getResources()[opmach]
            print(opname,"-",opmach)
        else:
            
            opname = oprinfo
            for rssname,myrs in dbmgr.getResources().items():
                for opr in myrs.getOperations():
                    if opr.getName()[:opr.getName().find("_")].strip() == opname:
                        myres = myrs
                        break
            if myres == None:
                if not opname in dbmgr.getResources():
                    myres = Resource(len(dbmgr.getResources()),"Machine",opname,16)
                    dbmgr.getResources()[opname]=myres
                else: 
                    myres = dbmgr.getResources()[opname]
                #print("Unrecognized:",oprinfo)
                
                
    
    optimestr = r['BoM Lines/Operations/Duration']
                    
    optime = 0
    
    if isinstance(optimestr, (int, float)):
        optime = float(1/60)*float(optimestr) # time assumed to be minute..

    newopr = Operation(len(dbmgr.getOperations()),opname+"_"+prod.getName(),optime)

    if myres !=None:
        myres.getOperations().append(newopr)
        newopr.getRequiredResources().append(myres)

    
    if not newopr.getName() in dbmgr.getOperations():
        dbmgr.getOperations()[newopr.getName()] = newopr
        
    return newopr

nopass = 0
for i,r in bom_df.iterrows():

    productrow = False
    if nopass > 0:
        nopass-=1
        continue
    
    if isinstance(r['Product'],str):
        if not pd.isnull(r['Product']):
            productrow= True
 
    #print("productrow ",productrow,"i",i)
  
    if productrow:
        newprod = MakeProduct(DataMgr,r['Product'])
        DataMgr.getProducts()[newprod.getName()] = newprod

        if not pd.isnull(r['BoM Lines/Component']):
            
            rawprod = MakeProduct(DataMgr,r['BoM Lines/Component'])

            quantity = r['BoM Lines/Quantity']

            newprod.getMPredecessors()[rawprod] = quantity
            newprod.getPredecessors().append(rawprod)
            rawprod.setSuccessor(newprod)

            DataMgr.getProducts()[rawprod.getName()] = rawprod
        

        j = i
        while j < len(bom_df):
            if not pd.isnull(bom_df.iloc[j,bom_df.columns.get_loc('Product')]) and j > i:
                break
            #print("Operation: ",bom_df.iloc[j,bom_df.columns.get_loc('BoM Lines/Operations')],"-",bom_df.iloc[j,bom_df.columns.get_loc('Reference')])

            if not pd.isnull(bom_df.iloc[j,bom_df.columns.get_loc('BoM Lines/Operations')]):
                myopr = MakeOperation(DataMgr,newprod,bom_df.iloc[j,bom_df.columns.get_loc('BoM Lines/Operations')])
                newprod.getOperations().append(myopr)

               
   
            j+=1 
        nopass=len(newprod.getOperations())-1
       
        i = j-1
        #print("i",i,"j",j)
    else:
        print(">>>>>>>>>>>> No Product definition: ", isinstance(r['Product'],str),pd.isnull(r['Product']))
        print("Product value: ",r['Product'])
        print(r)
        

print('Nr Products: ',len(DataMgr.getProducts()))
print('Nr Resources: ',len(DataMgr.getResources()))
print('Nr Operations: ',len(DataMgr.getOperations()))

bom_df.head(30)

DataMgr.getCustomerOrders().clear()

rel_path = "Sales Order Line (sale.order.line).xlsx"
abs_file_path = os.path.join(Path.cwd(), rel_path)
xls = pd.ExcelFile(abs_file_path)
orders_df = pd.read_excel(xls,"Sheet1")

#orders_df['Delivery Date']  = pd.to_datetime(orders_df['Delivery Date'] )

for i,r in orders_df.iterrows():
    code = r['Description']
    if code.find("[")>-1 and code.find("]")>-1:
        code = code[code.find("[")+1:code.find("]")]
    else:
        continue
    
    ordername = r['Order Reference']+"("+code+")"

    prodmatch = [prod for prod in DataMgr.getProducts().values() if prod.getPN() == code]

    if len(prodmatch) == 0:
        continue

    myprod = prodmatch[0]

    deadline = datetime.today().strftime('%Y-%m-%d')

    if not pd.isnull(r['Delivery Date']):
        deadline = r['Delivery Date'].strftime('%Y-%m-%d')
    

    #print(r['Delivery Date'],pd.isnull(r['Delivery Date']))
    myorder = CustomerOrder(len(DataMgr.getCustomerOrders()),ordername,myprod.getID()
                                                           ,myprod.getName(),r['Quantity'],str(deadline))
    myorder.setProduct(myprod)

    DataMgr.getCustomerOrders()[ordername] = myorder

print("Nr Customer Orders: ",len(DataMgr.getCustomerOrders()))
bom_df.head(25)